In [2]:
import funciones_creditos as fc
import numpy as np
import numpy_financial as npf
import pandas as pd
from sqlalchemy import create_engine

DATABASE_TYPE = 'postgresql'
USER = 'ricardo.torres'
PASSWORD = r'UW`bv&&rg>2!kwo\eOaD'
HOST = '34.123.217.240'
PORT = '5432'
DATABASE = 'credito'

database_uri = f"{DATABASE_TYPE}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
engine = create_engine(database_uri)

query = """
SELECT
    credits.id AS id_credito,
    credits.deposit_reference AS referencia_deposito,
    credits.amount AS saldo_inicial,
    credits.annual_interest_rate AS tasa_interes_anual,
    credits.payment_amount AS cuota_mensual,
    credits.term AS plazo,
    credits.operational_opening_date AS fecha_apertura,
    credits.first_payment_date AS fecha_primer_pago,
    CASE 
        WHEN NOT credits.open THEN LEAST(
            (SELECT MAX(payments_2.date) FROM payments AS payments_2 WHERE payments_2.credit_id = credits.id),
            credits.closing_date
        )
        ELSE NULL
    END AS fecha_cierre,
    payments.date AS fecha_pago,
    payments.amount AS monto_pago
FROM credits
    LEFT JOIN payments ON credits.id = payments.credit_id
WHERE
    credits.country = 'mx'
    AND credits.product_id IN (7, 8, 9, 10, 11)
ORDER BY credits.id, payments.date;
"""

df_creditos_pagos = pd.read_sql_query(query, engine)
engine.dispose()

fecha_columnas = ["fecha_apertura", "fecha_pago", "fecha_primer_pago", "fecha_cierre"]
df_creditos_pagos[fecha_columnas] = df_creditos_pagos[fecha_columnas].apply(pd.to_datetime, errors='coerce')





















In [3]:




IVA = 0.16  
FACTOR_IVA = 1 + IVA  
FECHA_CORTE = pd.Period("2021-12", freq='M')
df_creditos_pagos = df_creditos_pagos.sort_values(by=['id_credito', 'fecha_pago'])
df_creditos_pagos['fecha_anterior'] = df_creditos_pagos.groupby('id_credito')['fecha_pago'].shift(1)
df_creditos_pagos['fecha_anterior'] = df_creditos_pagos['fecha_anterior'].fillna(df_creditos_pagos['fecha_apertura'])
df_creditos_pagos['dias_transcurridos'] = (df_creditos_pagos['fecha_pago'] - df_creditos_pagos['fecha_anterior']).dt.days
df_creditos_pagos['monto_pagado_acumulado'] = df_creditos_pagos.groupby('id_credito')['monto_pago'].cumsum()
df_creditos_pagos['tasa_mensual'] = (df_creditos_pagos['tasa_interes_anual'] / 100) * FACTOR_IVA / 12
df_creditos_pagos['num_pagos_realizados'] = df_creditos_pagos['monto_pagado_acumulado'] / df_creditos_pagos['cuota_mensual']
df_creditos_pagos['saldo_posterior_valor_futuro'] = npf.fv(
    rate=df_creditos_pagos['tasa_mensual'], 
    nper=df_creditos_pagos['num_pagos_realizados'], 
    pmt=df_creditos_pagos['cuota_mensual'], 
    pv=-df_creditos_pagos['saldo_inicial'], 
    when=0  
)
df_creditos_pagos['saldo_posterior_valor_futuro'] = df_creditos_pagos['saldo_posterior_valor_futuro'].clip(lower=0)
df_creditos_pagos['saldo_posterior_intereses_acumulados'] = np.nan
creditos = df_creditos_pagos.groupby('id_credito').first()
for id_credito, grupo in df_creditos_pagos.groupby('id_credito'):
    saldo_actual = creditos.loc[id_credito, 'saldo_inicial']
    tasa_interes_anual = creditos.loc[id_credito, 'tasa_interes_anual'] / 100
    tasa_interes_diaria = tasa_interes_anual / 360
    intereses_acumulados = 0
    saldo_posterior_intereses_acumulados = []
    for _, renglon in grupo.iterrows():
        interes_generado = max(renglon['dias_transcurridos'] * tasa_interes_diaria * saldo_actual, 0)
        intereses_acumulados += interes_generado
        if renglon['monto_pago'] < intereses_acumulados * FACTOR_IVA:
            intereses_acumulados -= renglon['monto_pago'] / FACTOR_IVA
        else:
            saldo_actual -= renglon['monto_pago'] - (intereses_acumulados * FACTOR_IVA)
            intereses_acumulados = 0
        saldo_posterior_intereses_acumulados.append(saldo_actual)
    df_creditos_pagos.loc[grupo.index, 'saldo_posterior_intereses_acumulados'] = saldo_posterior_intereses_acumulados
df_creditos_pagos['mes_apertura'] = df_creditos_pagos['fecha_apertura'].dt.to_period('M')
df_creditos_pagos['saldo_final'] = np.where(
    df_creditos_pagos['mes_apertura'] >= FECHA_CORTE,
    df_creditos_pagos['saldo_posterior_intereses_acumulados'],
    df_creditos_pagos['saldo_posterior_valor_futuro']
)






In [4]:
df_creditos = df_creditos_pagos.groupby('id_credito').last()
fecha_inicial = pd.Timestamp('2024-09-30')
fecha_final = pd.Timestamp('2024-08-30')
df_creditos['paridad_inicial'] = fc.paridad(df_creditos, df_creditos_pagos, fecha_inicial)
df_creditos['paridad_final'] = fc.paridad(df_creditos, df_creditos_pagos, fecha_final)
df_creditos = fc.considerar(df_creditos, fecha_final)


# Obtener la matriz de roll over
matriz_roll_over = df_creditos.pivot_table('saldo_inicial', 'paridad_inicial', 'paridad_final', 'sum', 0)

# Normalizar para que cada fila sume 100%
matriz_roll_over_normalizada = matriz_roll_over.div(matriz_roll_over.sum(axis=1), axis=0) * 100

# Definir el orden correcto de filas y columnas
orden = ['Al Corriente', 'PAR 1', 'PAR 30', 'PAR 60', 'PAR 90', 'PAR 120', 'PAR 150', 'PAR 180', 'PAR 270', 'PAR 360']

# Reindexar tanto índices (filas) como columnas
matriz_roll_over_normalizada = matriz_roll_over_normalizada.reindex(index=orden, columns=orden)

matriz_roll_over_normalizada

paridad_final,Al Corriente,PAR 1,PAR 30,PAR 60,PAR 90,PAR 120,PAR 150,PAR 180,PAR 270,PAR 360
paridad_inicial,,,,,,,,,,
Al Corriente,98.863849,0.914248,0.087183,0.075992,0.020817,0.000000,0.000000,0.000000,0.000000,0.037912
PAR 1,46.201417,48.258515,4.563350,0.281549,0.083939,0.307949,0.000000,0.000000,0.000000,0.303281
PAR 30,11.645193,42.319861,38.990059,6.275430,0.574954,0.000000,0.000000,0.000000,0.000000,0.194503
PAR 60,0.739756,1.362518,56.588179,28.539578,11.982714,0.000000,0.000000,0.557424,0.229830,0.000000
PAR 90,0.000000,0.000000,3.751941,60.896713,28.120091,6.581655,0.406503,0.000000,0.000000,0.243096
PAR 120,0.000000,0.000000,0.000000,3.840879,74.283137,16.279857,4.715117,0.881010,0.000000,0.000000
PAR 150,0.000000,0.000000,0.000000,0.000000,4.057786,65.326932,25.099591,5.515691,0.000000,0.000000
PAR 180,0.000000,0.000000,0.000000,0.000000,0.000000,0.204477,28.871434,69.012884,1.911204,0.000000
PAR 270,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28.068499,71.440608,0.490893


In [8]:
import pandas as pd
import calendar
from datetime import date
from dateutil.relativedelta import relativedelta

# ----------------------------------------------------
# Función para obtener el último día de un mes
# ----------------------------------------------------
def last_day_of_month(year, month):
    """Devuelve el último día (entero) del mes especificado."""
    return calendar.monthrange(year, month)[1]

# ----------------------------------------------------
# Orden fijo de las paridades
# ----------------------------------------------------
orden = [
    'Al Corriente', 'PAR 1', 'PAR 30', 'PAR 60',
    'PAR 90', 'PAR 120', 'PAR 150', 'PAR 180',
    'PAR 270', 'PAR 360'
]

# ----------------------------------------------------
# Diccionario para determinar qué columnas sumar
# cuando hablamos de "bajada" o "mejoría"
# ----------------------------------------------------
columns_to_sum = {
    'Al Corriente': ['Al Corriente'],
    'PAR 1':        ['Al Corriente', 'PAR 1'],
    'PAR 30':       ['Al Corriente', 'PAR 1', 'PAR 30'],
    'PAR 60':       ['Al Corriente', 'PAR 1', 'PAR 30', 'PAR 60'],
    'PAR 90':       ['Al Corriente', 'PAR 1', 'PAR 30', 'PAR 60', 'PAR 90'],
    'PAR 120':      ['Al Corriente', 'PAR 1', 'PAR 30', 'PAR 60', 'PAR 90', 'PAR 120'],
    'PAR 150':      ['Al Corriente', 'PAR 1', 'PAR 30', 'PAR 60', 'PAR 90', 'PAR 120', 'PAR 150'],
    'PAR 180':      ['Al Corriente', 'PAR 1', 'PAR 30', 'PAR 60', 'PAR 90', 'PAR 120', 'PAR 150', 'PAR 180'],
    'PAR 270':      ['Al Corriente', 'PAR 1', 'PAR 30', 'PAR 60', 'PAR 90', 'PAR 120', 'PAR 150', 'PAR 180', 'PAR 270'],
    'PAR 360':      ['Al Corriente', 'PAR 1', 'PAR 30', 'PAR 60', 'PAR 90', 'PAR 120', 'PAR 150', 'PAR 180', 'PAR 270', 'PAR 360']
}

# ----------------------------------------------------
# DataFrame donde guardaremos la tabla resumen
# (una fila por paridad, una columna por mes)
# ----------------------------------------------------
summary_df = pd.DataFrame(index=orden)

# Diccionario para almacenar las matrices
all_matrices = {}

# ----------------------------------------------------
# Definir el rango de meses
# ----------------------------------------------------
start = date(2022, 1, 1)
end = date(2025, 2, 28)

current = start

while current <= end:
    # 1) Definir fecha_final: último día del mes actual
    year = current.year
    month = current.month
    day_final = last_day_of_month(year, month)
    fecha_final = pd.Timestamp(year, month, day_final)
    
    # 2) Definir fecha_inicial: último día del mes anterior
    prev_month = fecha_final - relativedelta(months=1)
    day_prev_final = last_day_of_month(prev_month.year, prev_month.month)
    fecha_inicial = pd.Timestamp(prev_month.year, prev_month.month, day_prev_final)
    
    # ------------------------------------------------
    # Asegurar que tu DF base se recalcule o filtre según necesites.
    # Aquí un ejemplo genérico:
    # ------------------------------------------------
    df_creditos = df_creditos_pagos.groupby('id_credito').last()
    
    # Calcular paridades
    df_creditos['paridad_inicial'] = fc.paridad(df_creditos, df_creditos_pagos, fecha_inicial)
    df_creditos['paridad_final']   = fc.paridad(df_creditos, df_creditos_pagos, fecha_final)
    
    df_creditos = fc.considerar(df_creditos, fecha_final)
    
    # ------------------------------------------------
    # Crear la matriz de rollover y normalizar
    # ------------------------------------------------
    matriz_roll_over = df_creditos.pivot_table(
        values='saldo_inicial',
        index='paridad_inicial',
        columns='paridad_final',
        aggfunc='sum',
        fill_value=0
    )
    
    # Normalizar por fila
    matriz_roll_over_normalizada = (
        matriz_roll_over
        .div(matriz_roll_over.sum(axis=1), axis=0) * 100
    )
    
    # Reindexar para que tenga siempre el mismo orden
    matriz_roll_over_normalizada = matriz_roll_over_normalizada.reindex(
        index=orden, 
        columns=orden
    )
    
    # ------------------------------------------------
    # Guardar la matriz en nuestro diccionario
    # ------------------------------------------------
    fecha_str = fecha_final.strftime('%Y-%m-%d')
    all_matrices[fecha_str] = matriz_roll_over_normalizada
    
    # ------------------------------------------------
    # Calcular la "bajada" para cada paridad
    # ------------------------------------------------
    sums_for_this_month = {}
    for row_paridad in orden:
        # columnas a sumar
        cols_bajada = columns_to_sum[row_paridad]
        # filtrar solo las que existan en la matriz
        cols_bajada = [c for c in cols_bajada if c in matriz_roll_over_normalizada.columns]
        
        # sumar
        sums_for_this_month[row_paridad] = matriz_roll_over_normalizada.loc[row_paridad, cols_bajada].sum()
    
    # Agregar columna al summary_df
    summary_df[fecha_str] = pd.Series(sums_for_this_month)
    
    # ------------------------------------------------
    # Avanzar al siguiente mes
    # ------------------------------------------------
    current = current + relativedelta(months=1)

# ----------------------------------------------------
# Ordenar cada renglón de summary_df y asignar ranking numérico
# ----------------------------------------------------
def sort_row_as_ranking(row):
    # Ordena de mayor a menor
    sorted_series = row.sort_values(ascending=False).reset_index(drop=True)
    # Renombra el índice para que sea 1, 2, 3, ... donde 1 es el mejor
    sorted_series.index = range(1, len(sorted_series) + 1)
    return sorted_series

summary_df_rank = summary_df.apply(sort_row_as_ranking, axis=1)

# ----------------------------------------------------
# Exportar a un único archivo Excel con tres hojas:
#  Hoja 1: Matrices
#  Hoja 2: Tabla resumen original (con fechas)
#  Hoja 3: Tabla resumen ordenada por ranking
# ----------------------------------------------------
with pd.ExcelWriter('resultado_rollover.xlsx') as writer:
    # Hoja 1: Todas las matrices, una debajo de la otra
    worksheet_name = 'Matrices'
    startrow = 0
    
    for fecha_str, matrix in all_matrices.items():
        matrix_title = f"Matriz Rollover Normalizada - {fecha_str}"
        matrix.to_excel(
            writer, 
            sheet_name=worksheet_name,
            startrow=startrow+1,  # Deja espacio para el título
            startcol=0
        )
        ws = writer.sheets[worksheet_name]
        ws.cell(row=startrow+1, column=1, value=matrix_title)
        startrow += len(matrix.index) + 4
    
    # Hoja 2: Tabla resumen original (con nombres de fecha)
    summary_df.to_excel(writer, sheet_name='Tabla_Resumen')
    
    # Hoja 3: Tabla resumen ordenada con ranking numérico (1 es el mejor)
    summary_df_rank.to_excel(writer, sheet_name='Tabla_Resumen_Ranking')


In [14]:
import pandas as pd
import numpy as np
import calendar
from datetime import date
from dateutil.relativedelta import relativedelta

# ----------------------------------------------------
# (Tu mismo código de arriba, sin cambios hasta summary_df_rank)
# ----------------------------------------------------

# ----------------------------------------------------
# Crear un DataFrame para guardar los 5 valores de linspace por fila
# ----------------------------------------------------
# Las columnas las llamaremos 'Linspace_1', 'Linspace_2', ..., 'Linspace_5'
linspace_df = pd.DataFrame(
    index=summary_df_rank.index,
    columns=[f"Linspace_{i}" for i in range(1, 6)]
)

# Recorremos cada fila de summary_df_rank para obtener
# el mejor valor (columna 1) y el valor #19 (columna 19).
# Luego generamos 5 puntos con np.linspace
for row_label in summary_df_rank.index:
    best_value = summary_df_rank.loc[row_label, 1]   # El mejor valor
    worst_value = summary_df_rank.loc[row_label, 19] # El valor #19
    
    # Generar 5 puntos equidistantes
    values = np.linspace(best_value, worst_value, 5)
    
    # Asignar esos 5 puntos al DataFrame linspace_df
    linspace_df.loc[row_label] = values

# ----------------------------------------------------
# Exportar a Excel (agregando una cuarta hoja)
# ----------------------------------------------------
with pd.ExcelWriter('resultado_rollover.xlsx') as writer:
    # Hoja 1: Todas las matrices, una debajo de la otra
    worksheet_name = 'Matrices'
    startrow = 0
    
    for fecha_str, matrix in all_matrices.items():
        matrix_title = f"Matriz Rollover Normalizada - {fecha_str}"
        matrix.to_excel(
            writer, 
            sheet_name=worksheet_name,
            startrow=startrow+1,  # Deja espacio para el título
            startcol=0
        )
        ws = writer.sheets[worksheet_name]
        ws.cell(row=startrow+1, column=1, value=matrix_title)
        startrow += len(matrix.index) + 4
    
    # Hoja 2: Tabla resumen original (con nombres de fecha)
    summary_df.to_excel(writer, sheet_name='Tabla_Resumen')
    
    # Hoja 3: Tabla resumen ordenada con ranking numérico (1 es el mejor)
    summary_df_rank.to_excel(writer, sheet_name='Tabla_Resumen_Ranking')

    # Hoja 4: DataFrame con los linspace (5 valores) de cada fila
    linspace_df.to_excel(writer, sheet_name='Linspace_Values')
